# BERTtopic

## Import libraries and data

In [ ]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

In [ ]:
data = pd.read_csv("../../data/processed/clean_reviews.csv", parse_dates=['Time'])
data.columns = [x.lower().replace(" ", "_") for x in data.columns ]
data.head()

## Prepare model

### Prepare embeddings

In [ ]:
# Prepare embeddings
sentence_model = SentenceTransformer("all-MiniLM-L12-v2")
embeddings = sentence_model.encode(data['cleaned_text'], show_progress_bar=True)

### UMAP for dimensionality reduction

In [ ]:

umap_model = UMAP(n_neighbors=100, 
                  n_components=3, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=4263)

### HDBSCAN for clustering

In [ ]:

hdbscan_model = HDBSCAN(min_cluster_size=50,
                        min_samples=20,
                        metric='euclidean',
                        prediction_data = True)

### Initialize BERTopic

In [ ]:
# Initiate BERTopic
topic_model = BERTopic(hdbscan_model=hdbscan_model,
                       umap_model=umap_model,
                       language="english",
                       calculate_probabilities=True,
                       nr_topics="auto")
# Run BERTopic model
topics, probabilities = topic_model.fit_transform(data['cleaned_text'], embeddings)

## Evaluate model

In [ ]:
# Get the list of topics
topic_model.get_topic_info()

In [ ]:
# Get top 10 terms for a topic
topic_model.get_topic(1)

In [ ]:
# Visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=20)

In [ ]:
# Visualize term rank decrease
topic_model.visualize_term_rank()

In [ ]:
# Visualize intertopic distance
topic_model.visualize_topics()

In [ ]:
# Visualize connections between topics using hierachical clustering
topic_model.visualize_hierarchy(top_n_topics=20)

In [ ]:
# Visualize similarity using heatmap
topic_model.visualize_heatmap()

In [ ]:
topic_model.visualize_documents(data["cleaned_text"], reduced_embeddings=umap_model.fit_transform(embeddings))

### Topics over time

In [ ]:
topics_over_time = topic_model.topics_over_time(data["cleaned_text"], data["time"])
topic_model.visualize_topics_over_time(topics_over_time)

### Topics by sentiment

In [ ]:
sentiment = [i for i in data["label"]]

In [ ]:
topics_by_sentiment = topic_model.topics_per_class(data["cleaned_text"], classes=sentiment)
topic_model.visualize_topics_per_class(topics_by_sentiment)

In [ ]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(data["cleaned_text"], calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(data["cleaned_text"][1], topic_token_distr[1])
df

## Append topic to dataset

In [ ]:
# Get the topic predictions
topic_prediction = topic_model.topics_[:]
# Save the predictions in the dataframe
data['topic_prediction'] = topic_prediction
data.head()